In [12]:
"""
    NN
"""
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [13]:
train = pd.read_csv('./train.csv')
test1 = pd.read_csv('./test1.csv')

features = train.drop(['Unnamed: 0','label'],axis = 1)
labels = train['label']

#### 构造新特征，特征比大于15的特征为关键特征

In [14]:
#数据探索，找到导致1的关键特征值
def find_key_feature(train,selected):
    temp = pd.DataFrame(columns = [0,1])
    temp[0] = train[train['label']==0][selected].value_counts()/len(train[train['label']==0]) 
    temp[1] = train[train['label']==1][selected].value_counts()/len(train[train['label']==1]) 
    temp[2] = temp[1]/temp[0]
    #选出大于15倍的特征
    result = temp[temp[2]>15].sort_values(2,ascending = False).index
    return result

key_features = {}
selected_col = ['osv','apptype','carrier','dev_height','dev_ppi','dev_width',
                'media_id','ntt','version','location','fea1_hash','cus_type']
for selected in selected_col:
    key_features[selected] = find_key_feature(train,selected)

#构造新特征，新特征字段 = 原始特征字段+1
def f1(x,selected):
    #判断是否在关键特征值里，是1，否0
    if x in key_features[selected]:
        return 1
    else:
        return 0

for selected in selected_col:
    if len(key_features[selected]) > 0:
        features[selected + '1'] = features[selected].apply(f1,args = (selected,))
        test1[selected + '1'] = test1[selected].apply(f1,args = (selected,))
        print(selected+'1 created')

osv1 created
apptype1 created
dev_height1 created
dev_ppi1 created
dev_width1 created
media_id1 created
ntt1 created
fea1_hash1 created


#### osv转换为float类型

In [15]:
#对osv进行数据清洗
import re
def f2(x):
    x = str(x)
    x = x.replace('Android_','')
    # 4.2.3.2是需要找到的字符
    result = re.match('[\d\.]+',x)
    if result:
        x = result.group()    #返回匹配结果
        if '.' in x:
            #把4.4.3转化成4.43
            #找到第一个小数点的位置
            index1 = x.index('.')
            if index1 > 0:
                #去掉所有小数点
                x = x.replace('.','')
                #加上原来第一个小数点
                x = float(x[0:index1]+'.'+x[index1:])
        else: x = float(x)
    else:
        x = 0
    #如果版本号过大，7930
    if x>1000:
        x = x/10000
    elif x>11:
        x = x/10
    return x
    
features['osv'] = features['osv'].apply(f2)
test1['osv'] = test1['osv'].apply(f2)

#### 特征筛选

In [16]:
#类别特征
cate_features = ['apptype','carrier','ntt','version','location','cus_type']
remove_list = ['os','lan','sid']
col = features.columns.tolist()
for i in remove_list:
    col.remove(i)
features = features[col]

#### 时间

In [17]:
import time  
from datetime import datetime  

def get_date(features):
    #除以1000，转换为日期格式
    features['timestamp'] = features['timestamp'].apply(lambda x:datetime.fromtimestamp(x/1000))
    
    #创建时间戳索引
    temp = pd.DatetimeIndex(features['timestamp'])
    features['year'] = temp.year
    features['month'] = temp.month
    features['day'] = temp.day
    features['week_day'] = temp.weekday
    features['hour'] = temp.hour
    features['minute'] = temp.minute

    #添加time_diff
    start_time = features['timestamp'].min()
    features['time_diff'] = features['timestamp'] - start_time
    
    #将time_diff转换为小时
    features['time_diff'] = features['time_diff'].dt.days * 24 +features['time_diff'].dt.seconds/3600
    
    #使用day，time_diff
    features.drop(['timestamp','year','month','minute','week_day'],axis = 1,inplace = True)
  
    return features

#对训练集提取时间多尺度
features = get_date(features)
#对测试集提取时间多尺度
test1 = get_date(test1)

#### LabelEncoder

In [18]:
#对lan进行特征编码LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#需要将训练集和测试集合并，然后统一做LabelEncoder
all_df = pd.concat([train,test1])
all_df['lan'] = all_df['lan'].astype('str')
all_df['lan'] = le.fit_transform(all_df['lan'])

#### 数据清洗

In [19]:
# 对于数值过大的异常值，设置为0
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

#对数据清洗，将V3=>3,V1=>1,V6=>6,V2=>2
#针对version，非数值类型 设置0
features['version'] = features['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
features['lan'] = all_df[all_df['label'].notnull()]['lan']

In [20]:
#测试集做预测,保持与features中的columns一致
test_fea = test1[features.columns]

test_fea['fea_hash'] = test_fea['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_fea['fea1_hash'] = test_fea['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

test_fea['version'] = test_fea['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
test_fea['lan'] = all_df[all_df['label'].isnull()]['lan']

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def ensemble_model(clf,train_x,train_y,test):
    #采用十折交叉验证
    sk = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 2021)
    prob = [] #记录最终结果
    mean_acc = 0 #记录平均准确率
    for k,(train_index,val_index) in enumerate(sk.split(train_x,train_y)):
        train_x_real = train_x.iloc[train_index]
        train_y_real = train_y.iloc[train_index]
        val_x = train_x.iloc[val_index]
        val_y = train_y.iloc[val_index]
        # 子模型训练
        clf = clf.fit(train_x_real,train_y_real)
        val_y_pred = clf.predict(val_x)
        #子模型评估
        acc_val = accuracy_score(val_y,val_y_pred)
        print('第{}个子模型 acc{}'.format(k+1,acc_val))
        mean_acc += acc_val/5
        #子模型预测0,1
        test_y_pred = clf.predict_proba(test)[:,-1] #soft得到概率值
        prob.append(test_y_pred)
    print(mean_acc)
    mean_prob = sum(prob)/5
    return mean_prob

In [24]:
#特征归一化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features1 = scaler.fit_transform(features)
test_fea1 = scaler.fit_transform(test_fea)

In [33]:
#使用神经网络进行融合
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

adam = Adam(lr = 0.001)
# 搭建模型
model = keras.Sequential([
    keras.layers.Dense(200, activation='relu', input_shape=[len(features.columns)] , kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.Dense(200, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.Dense(200, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.Dense(1, activation='sigmoid') 
])

#二分类任务
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=adam)
# batch_size 根据情况而选择，epochs可以一开始小一些
model.fit(features1, labels, batch_size=10240, epochs=100)

Epoch 1/100
49/49 [==============================] - 6s 107ms/step - loss: 0.8931 - accuracy: 0.7754
Epoch 2/100
49/49 [==============================] - 5s 107ms/step - loss: 0.5938 - accuracy: 0.8335
Epoch 3/100
49/49 [==============================] - 5s 110ms/step - loss: 0.4857 - accuracy: 0.8386
Epoch 4/100
49/49 [==============================] - 6s 124ms/step - loss: 0.4362 - accuracy: 0.8415
Epoch 5/100
49/49 [==============================] - 7s 137ms/step - loss: 0.4082 - accuracy: 0.8462
Epoch 6/100
49/49 [==============================] - 6s 121ms/step - loss: 0.3948 - accuracy: 0.8476
Epoch 7/100
49/49 [==============================] - 6s 118ms/step - loss: 0.3856 - accuracy: 0.8500
Epoch 8/100
49/49 [==============================] - 5s 112ms/step - loss: 0.3801 - accuracy: 0.8517
Epoch 9/100
49/49 [==============================] - 5s 108ms/step - loss: 0.3752 - accuracy: 0.8534
Epoch 10/100
49/49 [==============================] - 6s 115ms/step - loss: 0.3733 - accura

In [34]:
# 如果有验证集，可以进行设置
result = model.predict(test_fea1)

In [35]:
#保存结果
a = pd.DataFrame(test1['sid'])
a['label'] =  result
#转换为二分类
a['label'] = a['label'].apply(lambda x:0 if x<0.5 else 1)
a.to_csv('version8_NN100.csv',index = False)

In [36]:
a

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1
